In [1]:
import requests
from bs4 import BeautifulSoup as bs
import datetime as dt
import time
import json
import pymysql

In [2]:
def read_config(p:str) -> dict:
    """
    p: config file 경로
    """
    with open(p, 'r') as f:
        lines = f.readlines()
    config_dict={}

    for l in lines:
        idx = l.index('=')
        k = l[:idx]
        v = l[idx+1:]
        config_dict[k] = v.rstrip()
    
    return config_dict

In [3]:
baseballdb_config = read_config('./baseballdb_config')
conn = pymysql.connect(**baseballdb_config)

In [4]:
def bb_bat():
    now = dt.datetime.now()
    ny = now.strftime('%Y')
    nd = now.strftime('%Y%m%d')
    nm = now.strftime('%m')
    x = '4'
    result_lst = []
    for month in range(int(x.zfill(2)), int(nm)+1):
        url=f'https://sports.news.naver.com/kbaseball/schedule/index?date={nd}&month={month}&year={ny}&teamCode='
        response = requests.get(url)
        soup = bs(response.text, 'lxml')
        divs = soup.select('#calendarWrap>div')
        for div in divs:
            trs = div.select('tr')
            for tr in trs:
                tds = tr.select('td')
                if tr == trs[0]:
                    if len(tds) == 3:
                        break
                    elif tr.select('td.add_state'): 
                        fin_bat_name = ''
                        fin_bat_abs = ''
                        fin_bat_hthr = ''
                        fin_bat_bb = ''
                        fin_bat_kk = ''
                        fin_bat_rbi = ''
                        fin_bat_run = ''
                        fin_bat_ba = ''
                        fin_bat = (fin_bat_name, fin_bat_abs, fin_bat_hthr, fin_bat_bb, fin_bat_kk, fin_bat_rbi, fin_bat_run, fin_bat_ba)
                    elif tr.find_all('img')[2]['alt'] == "경기결과":
                        if tr.select('strong.td_score')[0].text.split(':')[0] == tr.select('strong.td_score')[0].text.split(':')[1]:
                            fin_bat_name = ''
                            fin_bat_abs = ''
                            fin_bat_hthr = ''
                            fin_bat_bb = ''
                            fin_bat_kk = ''
                            fin_bat_rbi = ''
                            fin_bat_run = ''
                            fin_bat_ba = ''
                            fin_bat = (fin_bat_name, fin_bat_abs, fin_bat_hthr, fin_bat_bb, fin_bat_kk, fin_bat_rbi, fin_bat_run, fin_bat_ba)
                        else:
                            raw_record_url = tr.select('a')[0]['href'].replace('game', 'games')
                            record_url = 'https://api-gw.sports.naver.com/schedule'+raw_record_url
                            record_rp = requests.get(record_url)
                            record = json.loads(record_rp.text)
                            bat_record = record['result']['recordData']['etcRecords']
                            fin_bat_name = bat_record[0]['result'].split('(')[0]
                            bat_score_record = record['result']['recordData']['battersBoxscore']
                            fin_bat_info = [bat_info for bat_info in list(bat_score_record.values())[2] if bat_info['name'] == fin_bat_name] or \
                                            [bat_info for bat_info in list(bat_score_record.values())[3] if bat_info['name'] == fin_bat_name]
                            for fb in fin_bat_info:
                                fin_bat_abs = str(fb['ab'])
                                fin_bat_hthr = str(fb['hit'])+'('+str(fb['hr'])+')'
                                fin_bat_bb = str(fb['bb'])
                                fin_bat_kk = str(fb['kk'])
                                fin_bat_rbi = str(fb['rbi'])
                                fin_bat_run = str(fb['run'])
                                fin_bat_ba = fb['hra']
                                fin_bat = (fin_bat_name, fin_bat_abs, fin_bat_hthr, fin_bat_bb, fin_bat_kk, fin_bat_rbi, fin_bat_run, fin_bat_ba)
                    else:
                        fin_bat_name = ''
                        fin_bat_abs = ''
                        fin_bat_hthr = ''
                        fin_bat_bb = ''
                        fin_bat_kk = ''
                        fin_bat_rbi = ''
                        fin_bat_run = ''
                        fin_bat_ba = ''
                        fin_bat = (fin_bat_name, fin_bat_abs, fin_bat_hthr, fin_bat_bb, fin_bat_kk, fin_bat_rbi, fin_bat_run, fin_bat_ba)
                else:
                    if len(tds) == 3:
                        break
                    if tr.select('td.add_state'): 
                        fin_bat_name = ''
                        fin_bat_abs = ''
                        fin_bat_hthr = ''
                        fin_bat_bb = ''
                        fin_bat_kk = ''
                        fin_bat_rbi = ''
                        fin_bat_run = ''
                        fin_bat_ba = ''
                        fin_bat = (fin_bat_name, fin_bat_abs, fin_bat_hthr, fin_bat_bb, fin_bat_kk, fin_bat_rbi, fin_bat_run, fin_bat_ba)
                    elif tr.find_all('img')[2]['alt'] == "경기결과":
                        if tr.select('strong.td_score')[0].text.split(':')[0] == tr.select('strong.td_score')[0].text.split(':')[1]:
                            fin_bat_name = ''
                            fin_bat_abs = ''
                            fin_bat_hthr = ''
                            fin_bat_bb = ''
                            fin_bat_kk = ''
                            fin_bat_rbi = ''
                            fin_bat_run = ''
                            fin_bat_ba = ''
                            fin_bat = (fin_bat_name, fin_bat_abs, fin_bat_hthr, fin_bat_bb, fin_bat_kk, fin_bat_rbi, fin_bat_run, fin_bat_ba)
                        else:
                            raw_record_url = tr.select('a')[0]['href'].replace('game', 'games')
                            record_url = 'https://api-gw.sports.naver.com/schedule'+raw_record_url
                            record_rp = requests.get(record_url)
                            record = json.loads(record_rp.text)
                            bat_record = record['result']['recordData']['etcRecords']
                            fin_bat_name = bat_record[0]['result'].split('(')[0]
                            bat_score_record = record['result']['recordData']['battersBoxscore']
                            fin_bat_info = [bat_info for bat_info in list(bat_score_record.values())[2] if bat_info['name'] == fin_bat_name] or \
                                            [bat_info for bat_info in list(bat_score_record.values())[3] if bat_info['name'] == fin_bat_name]
                            for fb in fin_bat_info:
                                fin_bat_abs = str(fb['ab'])
                                fin_bat_hthr = str(fb['hit'])+'('+str(fb['hr'])+')'
                                fin_bat_bb = str(fb['bb'])
                                fin_bat_kk = str(fb['kk'])
                                fin_bat_rbi = str(fb['rbi'])
                                fin_bat_run = str(fb['run'])
                                fin_bat_ba = fb['hra']
                                fin_bat = (fin_bat_name, fin_bat_abs, fin_bat_hthr, fin_bat_bb, fin_bat_kk, fin_bat_rbi, fin_bat_run, fin_bat_ba)
                    else:
                        fin_bat_name = ''
                        fin_bat_abs = ''
                        fin_bat_hthr = ''
                        fin_bat_bb = ''
                        fin_bat_kk = ''
                        fin_bat_rbi = ''
                        fin_bat_run = ''
                        fin_bat_ba = ''
                        fin_bat = (fin_bat_name, fin_bat_abs, fin_bat_hthr, fin_bat_bb, fin_bat_kk, fin_bat_rbi, fin_bat_run, fin_bat_ba)
                result_lst.append(fin_bat)
    with conn.cursor() as cursor:
        sql = "SELECT `GameID` FROM `TB_game`"
        cursor.execute(sql) 
        result=cursor.fetchall()
    ID_list=list(result)
    ID_list
    ordered_lst=[]
    for i in range(len(result_lst)):
        tem_lst=[ID_list[i]+result_lst[i]]
        ordered_lst.append(tem_lst)
    pit_lst = [x for sublist in ordered_lst for x in sublist]
    return pit_lst
bb_bat()

[(1, '이형종', '4', '1(0)', '0', '1', '1', '0', '0.250'),
 (2, '박성한', '4', '1(0)', '0', '0', '2', '0', '0.250'),
 (3, '알포드', '4', '4(1)', '0', '0', '3', '1', '1.000'),
 (4, '로하스', '6', '2(1)', '0', '1', '5', '2', '0.333'),
 (5, '박건우', '4', '2(0)', '0', '1', '1', '1', '0.500'),
 (6, '김휘집', '0', '0(0)', '1', '0', '1', '1', '0.000'),
 (7, '최형우', '1', '1(0)', '0', '0', '2', '0', '0.200'),
 (8, '이천웅', '1', '1(0)', '0', '0', '1', '0', '1.000'),
 (9, '한동희', '4', '1(0)', '0', '0', '2', '0', '0.091'),
 (10, '김지찬', '4', '1(0)', '1', '1', '1', '2', '0.375'),
 (11, '피렐라', '4', '1(1)', '0', '2', '2', '1', '0.182'),
 (12, '', '', '', '', '', '', '', ''),
 (13, '김현수', '4', '1(0)', '0', '0', '1', '1', '0.417'),
 (14, '김현수', '4', '1(0)', '0', '0', '1', '1', '0.417'),
 (15, '김인태', '4', '1(0)', '0', '0', '1', '0', '0.333'),
 (16, '', '', '', '', '', '', '', ''),
 (17, '', '', '', '', '', '', '', ''),
 (18, '', '', '', '', '', '', '', ''),
 (19, '', '', '', '', '', '', '', ''),
 (20, '김성욱', '4', '2(1)', '1',

In [5]:
bbt=bb_bat()

In [6]:
bbt

[(1, '이형종', '4', '1(0)', '0', '1', '1', '0', '0.250'),
 (2, '박성한', '4', '1(0)', '0', '0', '2', '0', '0.250'),
 (3, '알포드', '4', '4(1)', '0', '0', '3', '1', '1.000'),
 (4, '로하스', '6', '2(1)', '0', '1', '5', '2', '0.333'),
 (5, '박건우', '4', '2(0)', '0', '1', '1', '1', '0.500'),
 (6, '김휘집', '0', '0(0)', '1', '0', '1', '1', '0.000'),
 (7, '최형우', '1', '1(0)', '0', '0', '2', '0', '0.200'),
 (8, '이천웅', '1', '1(0)', '0', '0', '1', '0', '1.000'),
 (9, '한동희', '4', '1(0)', '0', '0', '2', '0', '0.091'),
 (10, '김지찬', '4', '1(0)', '1', '1', '1', '2', '0.375'),
 (11, '피렐라', '4', '1(1)', '0', '2', '2', '1', '0.182'),
 (12, '', '', '', '', '', '', '', ''),
 (13, '김현수', '4', '1(0)', '0', '0', '1', '1', '0.417'),
 (14, '김현수', '4', '1(0)', '0', '0', '1', '1', '0.417'),
 (15, '김인태', '4', '1(0)', '0', '0', '1', '0', '0.333'),
 (16, '', '', '', '', '', '', '', ''),
 (17, '', '', '', '', '', '', '', ''),
 (18, '', '', '', '', '', '', '', ''),
 (19, '', '', '', '', '', '', '', ''),
 (20, '김성욱', '4', '2(1)', '1',

In [7]:
with conn.cursor() as cursor:
    sql="INSERT INTO `TB_batter`(`GameID`, `Name`, `AtBats`, `Hit_Homerun`, `BB`, `StrikeOut`, `RBI`, `Run`, `BA`) VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s)"
    cursor.executemany(sql, bbt)
conn.commit()